In [26]:
import pandas as pd
from operator import attrgetter
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import numpy as np

In [27]:
customers = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-andrej-lebedev-nnw-021fd/Product_final_10/olist_customers_dataset.csv')
orders = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-andrej-lebedev-nnw-021fd/Product_final_10/olist_orders_dataset.csv')
items = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-andrej-lebedev-nnw-021fd/Product_final_10/olist_order_items_dataset.csv')

In [72]:
orders_copy.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_period,order_period_day,cohort,cohort_day,period_number_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10,2017-10-02,2017-10,2017-10-02,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,2018-07,2018-07-26,2018-07,2018-07-26,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,2018-08,2018-08-08,2018-08,2018-08-08,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,2017-11,2017-11-18,2017-11,2017-11-18,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2018-02,2018-02-13,2018-02,2018-02-13,0.0


In [28]:
orders_copy = orders.copy()

In [29]:
orders_copy['order_approved_at'].isna().sum()

160

In [30]:
orders_copy = orders_copy.dropna()

In [31]:
orders_copy.isna().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

In [73]:
orders_copy['order_purchase_timestamp'] = pd.to_datetime(orders_copy['order_purchase_timestamp'])

In [74]:
# создание колонок таблицы по месяцам и дням 
orders_copy['order_period'] = orders_copy['order_purchase_timestamp'].dt.to_period('M')
orders_copy['order_period_day'] = orders_copy['order_purchase_timestamp'].dt.to_period('D')

In [75]:
# создаю когорты по месяцу и дню 
orders_copy['cohort'] = orders_copy.groupby('customer_id')['order_purchase_timestamp'].transform('min').dt.to_period('M')
orders_copy['cohort_day'] = orders_copy.groupby('customer_id')['order_purchase_timestamp'].transform('min').dt.to_period('D')

In [76]:
# рассчитываю номер периода в днях 
orders_copy['period_number_month'] = np.floor((orders_copy.order_period_day - orders_copy.cohort_day).apply(attrgetter('n')))

In [77]:
orders_copy.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_period,order_period_day,cohort,cohort_day,period_number_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10,2017-10-02,2017-10,2017-10-02,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,2018-07,2018-07-24,2018-07,2018-07-24,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,2018-08,2018-08-08,2018-08,2018-08-08,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,2017-11,2017-11-18,2017-11,2017-11-18,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2018-02,2018-02-13,2018-02,2018-02-13,0.0


In [78]:
# агрегирую данные по когортам и периоду 
df_cohort = orders_copy.groupby(['cohort', 'period_number_month']).agg(n_customers=('customer_id', 'nunique')).reset_index()

In [79]:
df_cohort.head(200)

,cohort,period_number_month,n_customers
0,2016-09,0.0,1
1,2016-10,0.0,270
2,2016-12,0.0,1
3,2017-01,0.0,748
4,2017-02,0.0,1641
5,2017-03,0.0,2546
6,2017-04,0.0,2303
7,2017-05,0.0,3545
8,2017-06,0.0,3135
9,2017-07,0.0,3872


In [67]:
# делаю сводную для когорт 
cohort_pivot = df_cohort.pivot_table(index='cohort', columns='period_number_month', values='n_customers')

In [1]:
cohort_pivot.median()

NameError: name 'cohort_pivot' is not defined

In [69]:
cohort_size = cohort_pivot.iloc[:, 0]

In [70]:
retention_matrix = cohort_pivot.divide(cohort_size, axis=0)

In [71]:
retention_matrix.head()

period_number_month,0.0
cohort,
2016-09,1.0
2016-10,1.0
2016-12,1.0
2017-01,1.0
2017-02,1.0
